# Data 

Download from here: https://nlp.stanford.edu/data/glove.6B.zip (https://nlp.stanford.edu/projects/glove/)

# GloVe ANN benchmark (base / queries / ground truth)

This notebook loads Stanford GloVe embeddings and constructs a simple ANN benchmark for cosine (angular) similarity.

- **Parsing:** the file `glove.6B.*d.txt` is parsed into a token list `words` and an embedding matrix $E \in \mathbb{R}^{M \times D}$, where each row corresponds to a word vector.
- **Base and queries:** the search base is defined as the first $N$ vectors $X \in \mathbb{R}^{N \times D}$, and the query set is formed by sampling a random subset of rows from $X$ using indices $q_{\text{idx}}$: $Q = X[q_{\text{idx}}] \in \mathbb{R}^{n_q \times D}$.
- **Cosine / angular setting:** vectors are L2-normalized $X_n = \frac{X}{\|X\|_2}$ and $Q_n = \frac{Q}{\|Q\|_2}$, so cosine similarity reduces to a dot product $s(q, x) = \frac{q^\top x}{\|q\|_2\|x\|_2} = q_n^\top x_n$.
- **Ground truth (exact top-$k$):** exact neighbors are computed via batched matrix multiplication $S = Q_n X_n^\top$, excluding the trivial self-match (since queries are drawn from the base). The benchmark stores $\text{gt\_ids} \in \mathbb{N}^{n_q \times k}$ (top-$k$ neighbor indices) and $\text{gt\_scores} \in \mathbb{R}^{n_q \times k}$ (corresponding cosine similarity values).


In [1]:
import numpy as np
from typing import List, Tuple

def load_glove(path: str) -> Tuple[List[str], np.ndarray]:
    words: List[str] = []
    embeddings: List[List[float]] = []
    dim: int | None = None

    with open(path, "r", encoding="utf-8", errors="replace") as f:
        for line in f:
            p = line.split()
            if dim is None:
                dim = len(p) - 1
            words.append(p[0])
            embeddings.append([float(x) for x in p[1:]])

    X = np.asarray(embeddings, dtype=np.float32)

    # Sanity check: all rows must have the same dimensionality.
    if dim is not None:
        assert X.shape[1] == dim, f"Inconsistent embedding dimension: expected {dim}, got {X.shape[1]}"

    return words, X


def l2_normalize(X: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    X = X.astype(np.float32, copy=False)
    n = np.linalg.norm(X, axis=1, keepdims=True)
    return X / np.maximum(n, eps)


def make_base_and_queries(
    words: List[str],
    embeddings: np.ndarray,
    N: int = 100_000,
    nq: int = 10_000,
    seed: int = 0,
) -> Tuple[List[str], np.ndarray, np.ndarray]:
    assert 0 < N <= len(words), "N must be in (0, len(words)]."
    assert 0 < nq <= N, "nq must be in (0, N]."

    base_words = words[:N]
    X = embeddings[:N].astype(np.float32, copy=False)

    rng = np.random.default_rng(seed)
    q_idx = rng.choice(N, size=nq, replace=False).astype(np.int64)

    return base_words, X, q_idx


def ground_truth_topk_cosine(
    X: np.ndarray,
    q_idx: np.ndarray,
    k: int = 10,
    x_batch: int = 5_000,
    q_batch: int = 1_024,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    Xn = l2_normalize(X)
    Qn = Xn[q_idx]  # Queries are sampled from the base, already normalized.

    N = Xn.shape[0]
    nq = Qn.shape[0]
    assert 0 < k <= N, "k must be in (0, N]."

    best_scores = np.full((nq, k), -np.inf, dtype=np.float32)
    best_ids = np.full((nq, k), -1, dtype=np.int32)

    for x0 in range(0, N, x_batch):
        Xb = Xn[x0 : x0 + x_batch]
        xb = Xb.shape[0]
        kk = min(k, xb)

        for q0 in range(0, nq, q_batch):
            q1 = min(q0 + q_batch, nq)
            Qb = Qn[q0:q1]          # (qb, D)
            scores = Qb @ Xb.T      # (qb, xb)

            # Exclude trivial self-match (query vectors are from the base).
            q_base = q_idx[q0:q1]
            m = (q_base >= x0) & (q_base < x0 + xb)
            if m.any():
                rows = np.nonzero(m)[0]
                cols = (q_base[m] - x0).astype(np.int64)
                scores[rows, cols] = -np.inf

            idx = np.argpartition(scores, -kk, axis=1)[:, -kk:]
            sc = np.take_along_axis(scores, idx, axis=1)
            ids = (idx + x0).astype(np.int32)

            merged_sc = np.concatenate([best_scores[q0:q1], sc], axis=1)
            merged_ids = np.concatenate([best_ids[q0:q1], ids], axis=1)
            sel = np.argpartition(merged_sc, -k, axis=1)[:, -k:]

            best_scores[q0:q1] = np.take_along_axis(merged_sc, sel, axis=1)
            best_ids[q0:q1] = np.take_along_axis(merged_ids, sel, axis=1)

    order = np.argsort(best_scores, axis=1)[:, ::-1]
    best_scores = np.take_along_axis(best_scores, order, axis=1)
    best_ids = np.take_along_axis(best_ids, order, axis=1)

    return Xn, Qn, best_ids, best_scores


# --- Example usage ---
words, embeddings = load_glove("glove.6B/glove.6B.50d.txt")

N, nq, k = 100_000, 10_000, 10
base_words, X, q_idx = make_base_and_queries(words, embeddings, N=N, nq=nq, seed=42)
Xn, Qn, gt_ids, gt_scores = ground_truth_topk_cosine(X, q_idx, k=k, x_batch=5_000, q_batch=1_024)

print("Xn:", Xn.shape, "Qn:", Qn.shape, "gt_ids:", gt_ids.shape)
print("Example query word:", base_words[q_idx[0]])
print("Top neighbors:", [base_words[i] for i in gt_ids[0]])


Xn: (100000, 50) Qn: (10000, 50) gt_ids: (10000, 10)
Example query word: gleaner
Top neighbors: ['suara', 'telegraaf', 'prensa', 'jornal', 'andina', 'marca', 'republica', 'expresso', 'folha', 'ouest']
